## Prerequisites - Environment Setup

Before starting this workshop, make sure your .NET tooling and Azure credentials are in place.

### Step 1: Install .NET 9 SDK
Download the latest .NET 9 SDK from [https://dotnet.microsoft.com/download/dotnet/9.0](https://dotnet.microsoft.com/download/dotnet/9.0) and verify with `dotnet --version`.

### Step 2: Install the .NET Interactive extension
In VS Code install the **.NET Interactive Notebooks** extension so this notebook can execute C# cells.

### Step 3: Create `.env`
Copy the shared template and update it with your Azure resource identifiers:
```bash
# From Backend/dotnet/sk
copy ..\..\env.template .env       # Windows
cp ../../env.template .env         # macOS/Linux
```

Edit `.env` so it contains:
```bash
AZURE_OPENAI_ENDPOINT=https://<resource>.openai.azure.com
AZURE_OPENAI_API_KEY=<key>
AZURE_OPENAI_DEPLOYMENT_NAME=gpt-4o
AZURE_OPENAI_API_VERSION=2024-10-21

# Optional but required for Azure AI Foundry demos
PROJECT_ENDPOINT=https://<resource>.services.ai.azure.com/api/projects/<project>
PEOPLE_AGENT_ID=asst-people-agent
KNOWLEDGE_AGENT_ID=asst-knowledge-agent
MANAGED_IDENTITY_CLIENT_ID=<client-id-if-using-UMI>
```

**Important:** `.env` is ignored by Git. Treat it like a secret.

### Step 4: Select the .NET Interactive kernel
1. Click the kernel picker in the top-right of VS Code
2. Choose **".NET Interactive"**
3. Select the **C#** language

---

# .NET Semantic Kernel Agents Workshop

This notebook demonstrates modern AI agent development using Microsoft Semantic Kernel in .NET 9 with Azure OpenAI and Azure AI Foundry services.

## Workshop Focus Areas
1. **ChatCompletionAgent** – Build generic agents using the latest Semantic Kernel agent framework
2. **Azure AI Foundry Agents** – Connect to enterprise-grade agents hosted in Azure AI Projects
3. **Group Chat System** – Multi-agent collaboration with built-in orchestration

## Key Technologies
- **Semantic Kernel 1.65.0** – Microsoft's latest AI orchestration framework
- **Microsoft.SemanticKernel.Agents.AzureAI** – Native Azure AI Foundry agent integration
- **Azure OpenAI** – GPT-4 models for intelligent responses
- **NetEscapades.Configuration.Yaml** – Load shared YAML agent catalog

Let's build sophisticated AI agent systems using the latest Semantic Kernel patterns!

### Step 1: Verify .NET Environment

In [ ]:
// Step 1: Verify .NET Environment
// Check if .NET Interactive is working properly

using System;
using System.Runtime.InteropServices;

Console.WriteLine("=== .NET Environment Check ===");
Console.WriteLine($".NET Version: {Environment.Version}");
Console.WriteLine($"Runtime: {RuntimeInformation.FrameworkDescription}");
Console.WriteLine($"OS: {RuntimeInformation.OSDescription}");
Console.WriteLine($"Architecture: {RuntimeInformation.OSArchitecture}");
Console.WriteLine();
Console.WriteLine("✓ .NET Interactive is working!");
Console.WriteLine("Ready to install Semantic Kernel packages!");

// Return a value to ensure cell execution is visible
Console.WriteLine("Environment check completed successfully!");
"SUCCESS: .NET Interactive is running properly"

### Step 2: Install Semantic Kernel Packages

In [ ]:
// Step 2: Install Required NuGet Packages
// Install the same packages referenced by the API project so samples compile

Console.WriteLine("Installing Semantic Kernel + Azure packages ...");

#r "nuget: DotNetEnv, 3.1.1"
#r "nuget: NetEscapades.Configuration.Yaml, 3.1.0"

#r "nuget: Microsoft.SemanticKernel, 1.65.0"
#r "nuget: Microsoft.SemanticKernel.Abstractions, 1.65.0"
#r "nuget: Microsoft.SemanticKernel.Core, 1.65.0"
#r "nuget: Microsoft.SemanticKernel.Agents.Core, 1.65.0"
#r "nuget: Microsoft.SemanticKernel.Agents.Abstractions, 1.65.0"
#r "nuget: Microsoft.SemanticKernel.Agents.AzureAI, 1.65.0-preview"
#r "nuget: Microsoft.SemanticKernel.Connectors.AzureOpenAI, 1.65.0"
#r "nuget: Microsoft.SemanticKernel.Connectors.OpenAI, 1.65.0"

#r "nuget: Azure.AI.Projects, 1.0.0-beta.11"
#r "nuget: Azure.Identity, 1.16.0"
#r "nuget: Microsoft.Extensions.Configuration.Binder, 9.0.8"
#r "nuget: Microsoft.Extensions.DependencyInjection, 9.0.8"
#r "nuget: Microsoft.Extensions.Logging.Console, 9.0.8"

Console.WriteLine("\n✓ Packages restored. You're ready to import namespaces and create agents!");

### Troubleshooting: If You Don't See Output

**If the Run button does nothing or no output appears:**

#### 🚨 Most Common Issue: Kernel Not Selected
1. **Look at the top-right corner** of the notebook - you should see something like ".NET Interactive" or "C#"
2. **If you see "Select Kernel"** or a different kernel:
   - Click on it
   - Select **".NET Interactive"**
   - Choose **"C#"** from the sub-menu

#### 🔧 Other Solutions to Try:

1. **Install .NET Interactive Extension**:
   - Press `Ctrl+Shift+X` to open Extensions
   - Search for **".NET Interactive Notebooks"** by Microsoft
   - Install if not already installed
   - **Reload VS Code** after installation

2. **Restart Everything**:
   - Press `Ctrl+Shift+P`
   - Type "Developer: Reload Window"
   - Reopen the notebook and try again

3. **Check .NET Installation**:
   - Open a terminal (`Ctrl+Shift+``)
   - Run: `dotnet --version`
   - Should show .NET 8.0 or later

4. **Force Kernel Selection**:
   - Press `Ctrl+Shift+P`
   - Type "Notebook: Select Notebook Kernel"
   - Choose ".NET Interactive" → "C#"

**The cell below should show "42" as output if everything is working.**

### Step 3: Automatic `.env` Loading

This notebook mirrors the API startup sequence: it pulls configuration from `.env` (and falls back to defaults if entries are missing).

**How it works:**
1. Searches for a `.env` file in the current folder or parents
2. Loads all environment variables via `DotNetEnv`
3. Demonstrates how the API boots using YAML + env overrides

**Sample `.env`:**
```
AZURE_OPENAI_ENDPOINT=https://your-resource.openai.azure.com
AZURE_OPENAI_API_KEY=your_api_key_here
AZURE_OPENAI_DEPLOYMENT_NAME=gpt-4o
AZURE_OPENAI_API_VERSION=2024-10-21

# Optional but required for Azure AI Foundry agent demos
PROJECT_ENDPOINT=https://your-resource.services.ai.azure.com/api/projects/your-project
PEOPLE_AGENT_ID=asst-your-people-agent-id
KNOWLEDGE_AGENT_ID=asst-your-knowledge-agent-id
MANAGED_IDENTITY_CLIENT_ID=<client-id-if-using-UMI>
```

If the Foundry values are omitted the notebook will still run but those agent samples will be skipped. The API behaves the same way.

In [ ]:
// Step 3: Import Required Namespaces
// Import all the Semantic Kernel namespaces for modern agent development

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Agents;
using Microsoft.SemanticKernel.Agents.OpenAI;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.Extensions.Logging;
using Microsoft.Extensions.DependencyInjection;
using System.ComponentModel;
using System.IO;
using DotNetEnv;

Console.WriteLine("✓ All namespaces imported successfully!");
Console.WriteLine("Ready to configure Azure OpenAI and create modern agents!");

### Step 4: Configure Azure OpenAI

In [ ]:
// Step 4: Configure Azure OpenAI Connection
// Automatically load configuration from .env file

Console.WriteLine("Configuring Azure OpenAI...");

// Load .env file from current directory or parent directories
try
{
    // Try loading .env from current notebook directory
    var notebookDir = Directory.GetCurrentDirectory();
    var envFile = Path.Combine(notebookDir, ".env");
    
    if (File.Exists(envFile))
    {
        Env.Load(envFile);
        Console.WriteLine($"✓ Loaded .env file from: {envFile}");
    }
    else
    {
        // Try parent directories (common pattern)
        var parentDir = Directory.GetParent(notebookDir);
        while (parentDir != null)
        {
            envFile = Path.Combine(parentDir.FullName, ".env");
            if (File.Exists(envFile))
            {
                Env.Load(envFile);
                Console.WriteLine($"✓ Loaded .env file from: {envFile}");
                break;
            }
            parentDir = parentDir.Parent;
        }
        
        if (parentDir == null)
        {
            Console.WriteLine("! No .env file found, using system environment variables");
        }
    }
}
catch (Exception ex)
{
    Console.WriteLine($"! Error loading .env file: {ex.Message}");
    Console.WriteLine("Falling back to system environment variables");
}

// Load configuration from environment variables (now populated from .env)
var azureOpenAIEndpoint = Environment.GetEnvironmentVariable("AZURE_OPENAI_ENDPOINT");
var azureOpenAIApiKey = Environment.GetEnvironmentVariable("AZURE_OPENAI_API_KEY");
var deploymentName = Environment.GetEnvironmentVariable("AZURE_OPENAI_DEPLOYMENT_NAME") ?? "gpt-4o";

// Required environment variables
var requiredVars = new Dictionary<string, string?>
{
    { "AZURE_OPENAI_ENDPOINT", azureOpenAIEndpoint },
    { "AZURE_OPENAI_API_KEY", azureOpenAIApiKey },
    { "AZURE_OPENAI_DEPLOYMENT_NAME", deploymentName }
};

// Check if all required variables are present
var missingVars = requiredVars.Where(kvp => string.IsNullOrWhiteSpace(kvp.Value)).Select(kvp => kvp.Key).ToList();

if (missingVars.Any())
{
    Console.WriteLine("❌ Missing required Azure OpenAI configuration!");
    Console.WriteLine($"Missing variables: {string.Join(", ", missingVars)}");
    Console.WriteLine();
    Console.WriteLine("To fix this, create a .env file in this directory (or parent directory) with:");
    Console.WriteLine("AZURE_OPENAI_ENDPOINT=https://your-resource.openai.azure.com/");
    Console.WriteLine("AZURE_OPENAI_API_KEY=your_api_key_here");
    Console.WriteLine("AZURE_OPENAI_DEPLOYMENT_NAME=gpt-4o");
    Console.WriteLine();
    Console.WriteLine("Then restart this notebook kernel and run this cell again.");
    throw new InvalidOperationException("Required Azure OpenAI configuration missing!");
}

Console.WriteLine("✓ Azure OpenAI configuration loaded successfully!");
Console.WriteLine($"Endpoint: {azureOpenAIEndpoint}");
Console.WriteLine($"Deployment: {deploymentName}");
Console.WriteLine("Ready to create Semantic Kernel!");

### Step 5: Create Semantic Kernel Instance

In [ ]:
// Step 5: Create Semantic Kernel Instance
// Set up the Semantic Kernel with Azure OpenAI chat completion

Console.WriteLine("Creating Semantic Kernel...");

// Create Kernel Builder
var builder = Kernel.CreateBuilder();

// Add Azure OpenAI Chat Completion service
builder.AddAzureOpenAIChatCompletion(
    deploymentName: deploymentName,
    endpoint: azureOpenAIEndpoint,
    apiKey: azureOpenAIApiKey);

// Add logging
builder.Services.AddLogging(c => c.AddConsole().SetMinimumLevel(LogLevel.Information));

// Build the kernel
var kernel = builder.Build();

Console.WriteLine("✓ Semantic Kernel created successfully!");
Console.WriteLine("Ready to create our first agent!");

In [ ]:
// Step 6: Create Your First ChatCompletionAgent
// This demonstrates the modern Semantic Kernel agent pattern

Console.WriteLine("Creating ChatCompletionAgent...");

// Create a generic ChatCompletionAgent
var genericAgent = new ChatCompletionAgent()
{
    Name = "GenericAssistant",
    Instructions = @"You are a helpful AI assistant that can answer questions and have conversations.
    
    Your capabilities include:
    - Answering general questions
    - Helping with problem-solving
    - Providing explanations and information
    - Having natural conversations
    
    Always be helpful, accurate, and professional in your responses.",
    Kernel = kernel
};

Console.WriteLine("✓ ChatCompletionAgent created successfully!");
Console.WriteLine($"Agent Name: {genericAgent.Name}");
Console.WriteLine($"Agent Type: {genericAgent.GetType().Name}");
Console.WriteLine("Ready to test the modern agent pattern!");

### Test Your First Agent

In [ ]:
// Test the ChatCompletionAgent
// Send a message to the agent and see its response

Console.WriteLine("Testing the ChatCompletionAgent...");

// Create a chat history for the conversation
var chatHistory = new ChatHistory();

// Test message - you can modify this to ask different questions
var userMessage = "Hello! What is artificial intelligence and how does it work?";

Console.WriteLine($"User: {userMessage}");
chatHistory.AddUserMessage(userMessage);

try
{
    // Get agent response using the modern agent pattern
    await foreach (var response in genericAgent.InvokeAsync(chatHistory))
    {
        Console.WriteLine($"Agent ({genericAgent.Name}): {response.Content}");
        // Add the agent's response to chat history for context
        chatHistory.Add(response);
    }
    
    Console.WriteLine();
    Console.WriteLine("✓ ChatCompletionAgent test completed successfully!");
    Console.WriteLine("You can modify the userMessage above to test different questions.");
}
catch (Exception ex)
{
    Console.WriteLine($"❌ Error testing agent: {ex.Message}");
    Console.WriteLine("Please check your Azure OpenAI configuration.");
}

### Create Configuration-Based Agents

Now let's explore the modern approach using configuration files! This demonstrates how to use the `config.yml` file to manage agent instructions and settings, making the system more maintainable and consistent across different frameworks (Python and .NET).

In [ ]:
// Load Configuration-Based Agents
// This demonstrates the modern approach using config.yml for agent instructions

Console.WriteLine("Loading agents from configuration...");

// First, let's load our YAML configuration
#r "nuget: NetEscapades.Configuration.Yaml, 3.1.0"
#r "nuget: Microsoft.Extensions.Configuration, 9.0.8"
#r "nuget: Microsoft.Extensions.Configuration.Binder, 9.0.8"

using Microsoft.Extensions.Configuration;
using NetEscapades.Configuration.Yaml;

// Build configuration from config.yml file
var configuration = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddYamlFile("config.yml", optional: false, reloadOnChange: true)
    .Build();

Console.WriteLine("✓ Configuration loaded from config.yml");

// Helper method to get agent instructions from config
string GetAgentInstructions(string agentName, string fallbackInstructions)
{
    var instructions = configuration[$"agents:{agentName}:instructions"];
    return !string.IsNullOrEmpty(instructions) ? instructions : fallbackInstructions;
}

string GetAgentDescription(string agentName, string fallbackDescription)
{
    var description = configuration[$"agents:{agentName}:metadata:description"];
    return !string.IsNullOrEmpty(description) ? description : fallbackDescription;
}

Console.WriteLine("✓ Configuration helper methods ready");
Console.WriteLine("Now we can create agents with instructions loaded from config.yml!");

In [ ]:
// Create Configuration-Based Agents
// These agents load their instructions from config.yml, demonstrating modern best practices

Console.WriteLine("Creating configuration-based agents...");

// People Lookup Agent - Instructions loaded from config.yml
var peopleAgentInstructions = GetAgentInstructions("people_lookup", 
    @"You are a People Lookup Agent, an expert at finding and providing information about people, contacts, and team members.");
var peopleAgentDescription = GetAgentDescription("people_lookup", 
    "Expert at finding information about people, contacts, and team members");

var peopleAgent = new ChatCompletionAgent()
{
    Name = "PeopleLookupAgent",
    Instructions = peopleAgentInstructions,
    Kernel = kernel
};

// Knowledge Finder Agent - Instructions loaded from config.yml
var knowledgeAgentInstructions = GetAgentInstructions("knowledge_finder", 
    @"You are a Knowledge Finder Agent, a specialist at searching, retrieving, and organizing information from various knowledge sources.");
var knowledgeAgentDescription = GetAgentDescription("knowledge_finder", 
    "Specialist at searching and retrieving relevant knowledge, documents, and information");

var knowledgeAgent = new ChatCompletionAgent()
{
    Name = "KnowledgeFinderAgent", 
    Instructions = knowledgeAgentInstructions,
    Kernel = kernel
};

// Generic Agent - Instructions loaded from config.yml
var genericAgentInstructions = GetAgentInstructions("generic_agent", 
    @"You are a helpful, knowledgeable, and versatile assistant powered by Azure OpenAI.");
var genericAgentDescription = GetAgentDescription("generic_agent", 
    "General-purpose conversational agent powered by Azure OpenAI for various tasks and inquiries");

var genericAgent = new ChatCompletionAgent()
{
    Name = "GenericAgent",
    Instructions = genericAgentInstructions,
    Kernel = kernel
};

Console.WriteLine("✓ Configuration-based agents created successfully!");
Console.WriteLine($"1. {peopleAgent.Name} - {peopleAgentDescription}");
Console.WriteLine($"2. {knowledgeAgent.Name} - {knowledgeAgentDescription}");
Console.WriteLine($"3. {genericAgent.Name} - {genericAgentDescription}");
Console.WriteLine();
Console.WriteLine("💡 Key Benefits of Configuration-Based Approach:");
Console.WriteLine("   • Instructions centralized in config.yml");
Console.WriteLine("   • Consistent across Python and .NET implementations");
Console.WriteLine("   • Easy to update without recompiling code");
Console.WriteLine("   • Better maintainability and version control");
Console.WriteLine();
Console.WriteLine("These agents now use the enhanced instructions from config.yml!");
Console.WriteLine();
Console.WriteLine("These agents demonstrate specialized capabilities that could integrate with Azure AI Foundry services!");

In [ ]:
// Test Azure AI Foundry Agents
// Let's test each specialized agent with questions in their area of expertise

Console.WriteLine("Testing Azure AI Foundry specialized agents...");
Console.WriteLine();

// Test cases for each agent
var testCases = new[]
{
    new { Agent = peopleAgent, Question = "I need to find someone who can help with .NET development. Who should I contact?" },
    new { Agent = knowledgeAgent, Question = "What are the best practices for implementing microservices architecture?" }
};

for (int i = 0; i < testCases.Length; i++)
{
    var testCase = testCases[i];
    
    try
    {
        Console.WriteLine($"Test {i + 1}: {testCase.Agent.Name}");
        Console.WriteLine($"Question: {testCase.Question}");
        Console.WriteLine(new string('-', 50));
        
        // Create chat history for this test
        var testChatHistory = new ChatHistory();
        testChatHistory.AddUserMessage(testCase.Question);
        
        // Get response from the agent
        await foreach (var response in testCase.Agent.InvokeAsync(testChatHistory))
        {
            Console.WriteLine($"Response: {response.Content}");
        }
        
        Console.WriteLine(new string('=', 70));
        Console.WriteLine();
    }
    catch (Exception ex)
    {
        Console.WriteLine($"❌ Error testing {testCase.Agent.Name}: {ex.Message}");
        Console.WriteLine(new string('=', 70));
        Console.WriteLine();
    }
}

Console.WriteLine("✓ Specialized agent testing completed!");
Console.WriteLine("Each agent demonstrated expertise in their specialized domain.");

### Create Group Chat with All Three Agents

In [ ]:
// Create Group Chat with Modern AgentGroupChat
// This demonstrates multi-agent collaboration using Semantic Kernel's built-in group chat

Console.WriteLine("Creating Agent Group Chat...");

// Create an AgentGroupChat with all three agents
var groupChat = new AgentGroupChat(genericAgent, peopleAgent, knowledgeAgent)
{
    // Configure the execution settings for the group chat
    ExecutionSettings = new OpenAIPromptExecutionSettings()
    {
        ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions,
        Temperature = 0.7,
        MaxTokens = 1000
    }
};

Console.WriteLine("✓ Agent Group Chat created successfully!");
Console.WriteLine($"Participants: {string.Join(", ", new[] { genericAgent.Name, peopleAgent.Name, knowledgeAgent.Name })}");
Console.WriteLine("Ready for multi-agent collaboration!");

// Test the group chat with a complex question that benefits from multiple perspectives
var complexQuestion = @"We're launching a new AI-powered project management tool. 
We need to understand:
1. Who should be involved in the development team?
2. What technical knowledge and best practices should we consider?
3. How can we ensure the project is successful?

Please provide a comprehensive response with insights from different perspectives.";

Console.WriteLine();
Console.WriteLine("Testing Group Chat with Complex Question:");
Console.WriteLine($"Question: {complexQuestion}");
Console.WriteLine(new string('=', 80));

try
{
    // Add the user question to the group chat
    groupChat.AddChatMessage(new ChatMessageContent(AuthorRole.User, complexQuestion));
    
    Console.WriteLine("Starting group chat collaboration...");
    
    // Track the conversation
    int messageCount = 0;
    const int maxMessages = 6; // Limit to prevent too long conversations
    
    // Process the group chat conversation
    await foreach (var message in groupChat.InvokeAsync())
    {
        messageCount++;
        var author = message.AuthorName ?? "Unknown";
        
        Console.WriteLine($"\n[{messageCount}] {author}:");
        Console.WriteLine($"{message.Content}");
        Console.WriteLine(new string('-', 60));
        
        // Break if we've reached the message limit
        if (messageCount >= maxMessages)
        {
            Console.WriteLine("\n[Conversation limit reached for demo purposes]");
            break;
        }
    }
    
    Console.WriteLine();
    Console.WriteLine("✓ Group Chat completed successfully!");
    Console.WriteLine($"Generated {messageCount} messages from multiple agents working together!");
}
catch (Exception ex)
{
    Console.WriteLine($"❌ Error in group chat: {ex.Message}");
    Console.WriteLine("Please check your Azure OpenAI configuration and try again.");
}

### Test Group Chat System

In [ ]:
// Try Your Own Group Chat Question
// Modify the question below to explore different multi-agent scenarios

Console.WriteLine("Custom Group Chat Test");
Console.WriteLine("Feel free to modify the question below to explore different topics!");
Console.WriteLine();

// MODIFY THIS QUESTION to test different scenarios:
var customQuestion = @"How can we improve remote team collaboration? 
Consider finding the right people, knowledge management, and general best practices.";

Console.WriteLine($"Your Question: {customQuestion}");
Console.WriteLine(new string('=', 60));

try
{
    // Create a new group chat for this custom question
    var customGroupChat = new AgentGroupChat(genericAgent, peopleAgent, knowledgeAgent);
    
    // Add the custom question
    customGroupChat.AddChatMessage(new ChatMessageContent(AuthorRole.User, customQuestion));
    
    Console.WriteLine("Starting custom group chat...");
    
    // Track the conversation
    int messageCount = 0;
    const int maxMessages = 4; // Shorter for custom test
    
    // Process the group chat conversation
    await foreach (var message in customGroupChat.InvokeAsync())
    {
        messageCount++;
        var author = message.AuthorName ?? "Unknown";
        
        Console.WriteLine($"\n[{messageCount}] {author}:");
        Console.WriteLine($"{message.Content}");
        Console.WriteLine(new string('-', 40));
        
        // Break if we've reached the message limit
        if (messageCount >= maxMessages)
        {
            Console.WriteLine("\n[Demo limit reached]");
            break;
        }
    }
    
    Console.WriteLine();
    Console.WriteLine("✓ Custom group chat completed!");
    Console.WriteLine("Feel free to modify the customQuestion above to explore different topics!");
}
catch (Exception ex)
{
    Console.WriteLine($"❌ Error in custom group chat: {ex.Message}");
}

## Workshop Summary

Congratulations! You've successfully built modern AI agents using the latest Semantic Kernel patterns:

### What You've Learned:

1. **ChatCompletionAgent** - The modern way to create conversational agents in Semantic Kernel
   - Simple, clean API for agent creation
   - Built-in conversation management
   - Easy integration with Azure OpenAI

2. **Azure AI Foundry Agents** - Specialized agents with expert knowledge
   - PeopleLookupAgent for finding contacts and team information
   - KnowledgeFinderAgent for research and information retrieval
   - Focused expertise with clear role definitions

3. **AgentGroupChat** - Multi-agent collaboration made simple
   - Built-in orchestration and coordination
   - Natural conversation flow between agents
   - Automatic turn-taking and response synthesis

### Key Benefits of This Approach:
- ✅ **Modern Pattern** - Uses Semantic Kernel 1.65.0 and the latest agents APIs
- ✅ **Production Ready** - Same patterns used in the backend API
- ✅ **Simplified Code** - Less boilerplate, more functionality
- ✅ **Azure Integration** - Seamless Azure OpenAI connectivity

### Next Steps:
- Explore the backend API at `/Backend/dotnet/sk/`
- Try different agent instructions and personalities
- Experiment with more complex group chat scenarios
- Build your own specialized agents for specific domains

You now have the foundation to build sophisticated multi-agent AI systems!